In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os, io

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import BinaryFocalCrossentropy
import tensorflow.keras.backend as K

import datetime
import matplotlib.pyplot as plt

In [2]:
BATCH = 16
RESIZE = 128

In [3]:
# Define Dice coefficient metric
def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_f = K.flatten(K.cast(y_true, 'float32'))
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Define Dice loss
def dice_loss(y_true, y_pred):
    bce = BinaryFocalCrossentropy(alpha=0.9)(y_true, y_pred)
    return (1 - dice_coefficient(y_true, y_pred)) + bce



def load_tif_data(data_dir):
    images, masks = [], []
    for patient_dir in os.listdir(data_dir):
        patient_path = os.path.join(data_dir, patient_dir)
        if not os.path.isdir(patient_path):
            continue
        # Get all .tif files in the patient directory
        tif_files = [f for f in os.listdir(patient_path) if f.endswith('.tif') and 'mask' not in f]
        for img_file in tif_files:
            img_path = os.path.join(patient_path, img_file)
            images.append(img_path)
            # Construct corresponding mask filename
            mask_file = img_file.replace('.tif', '_mask.tif')
            mask_path = os.path.join(patient_path, mask_file)
            masks.append(mask_path)
            if not os.path.exists(mask_path):
                continue  # Skip if mask file doesn’t exist
    
    return pd.DataFrame({'img': images, 'mask': masks})

In [4]:
# data_dir = 'D:/Github/Brain Tumor MRI Segmentation/dataset/'  # Update with your dataset path
# df = load_tif_data(data_dir)
df = pd.read_csv('data_addrs.csv')

In [5]:
df

,img,mask
0,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
1,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
2,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
4,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
...,...,...
3924,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3925,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3926,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...
3927,D:/Github/Brain Tumor MRI Segmentation/dataset...,D:/Github/Brain Tumor MRI Segmentation/dataset...


In [6]:
x_tr, x_val = train_test_split(df, test_size=0.3, random_state=42)
x_val, x_ts = train_test_split(x_val, test_size=0.5, random_state=42)

print(f"Train: {len(x_tr)}, Val: {len(x_val)}, Test: {len(x_ts)}")

Train: 2750, Val: 589, Test: 590


In [7]:
def _load_tif_image(path, ch):
    img = Image.open(path.decode())
    
    if int(ch) == 3:
        img = img.convert('RGB')
    else:
        img = img.convert('L')
    
    img = img.resize((RESIZE, RESIZE))
    img = np.array(img).astype(np.float32) / 255.0

    # If grayscale (2D), expand to [H, W, 1]
    if img.ndim == 2:
        img = np.expand_dims(img, axis=-1)
    return img


def load_image_and_mask(image_path, mask_path):
    # Load image
    image = tf.numpy_function(_load_tif_image, [image_path, 3], tf.float32)
    image.set_shape([RESIZE, RESIZE, 3])  # or [RESIZE, RESIZE, 1]

    # Load mask (assume single channel)
    mask = tf.numpy_function(_load_tif_image, [mask_path, 1], tf.float32)
    mask.set_shape([RESIZE, RESIZE, 1])

    return image, mask

In [8]:
trds = tf.data.Dataset.from_tensor_slices((x_tr['img'], x_tr['mask']))
vlds = tf.data.Dataset.from_tensor_slices((x_val['img'], x_val['mask']))
tsds = tf.data.Dataset.from_tensor_slices((x_ts['img'], x_ts['mask']))


trds = (trds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

vlds = (vlds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

tsds = (tsds
	.shuffle(1024)
	.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
	.cache()
	.batch(BATCH)
	.prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# for a, b in tsds.take(1):
#     print(a)
#     print()
#     print(b)
    

tf.Tensor(
[[[[0.         0.         0.        ]
   [0.         0.         0.        ]
   [0.         0.         0.        ]
   ...
   [0.00392157 0.00392157 0.00392157]
   [0.         0.         0.        ]
   [0.         0.         0.        ]]

  [[0.         0.         0.        ]
   [0.01176471 0.01176471 0.01176471]
   [0.00392157 0.00392157 0.00392157]
   ...
   [0.00784314 0.00784314 0.00784314]
   [0.01176471 0.01176471 0.01176471]
   [0.01568628 0.01568628 0.01568628]]

  [[0.00392157 0.00392157 0.00392157]
   [0.01568628 0.01568628 0.01568628]
   [0.01568628 0.01568628 0.01568628]
   ...
   [0.01176471 0.01176471 0.01176471]
   [0.02352941 0.02352941 0.02352941]
   [0.01568628 0.01568628 0.01568628]]

  ...

  [[0.00784314 0.00784314 0.00784314]
   [0.01960784 0.01960784 0.01960784]
   [0.01568628 0.01568628 0.01568628]
   ...
   [0.01568628 0.01568628 0.01568628]
   [0.01176471 0.01176471 0.01176471]
   [0.01176471 0.01176471 0.01176471]]

  [[0.00392157 0.00392157 0.003921

In [10]:
# Define TumorSegNet (same as before, included for completeness)
def TumorSegNet(input_shape=(256, 256, 3)):
    inputs = Input(input_shape, name='Input_Layer')
    # Encoder
    c1 = Conv2D(64, 3, activation='relu', padding='same', name='Conv1_1')(inputs)
    c1 = Conv2D(64, 3, activation='relu', padding='same', name='Conv1_2')(c1)
    p1 = MaxPooling2D((2, 2), name='MaxPool1')(c1)
    c2 = Conv2D(128, 3, activation='relu', padding='same', name='Conv2_1')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same', name='Conv2_2')(c2)
    p2 = MaxPooling2D((2, 2), name='MaxPool2')(c2)
    c3 = Conv2D(256, 3, activation='relu', padding='same', name='Conv3_1')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same', name='Conv3_2')(c3)
    p3 = MaxPooling2D((2, 2), name='MaxPool3')(c3)
    # Bottleneck
    c4 = Conv2D(512, 3, activation='relu', padding='same', name='Bottleneck_Conv1')(p3)
    c4 = Conv2D(512, 3, activation='relu', padding='same', name='Bottleneck_Conv2')(c4)
    c4 = Dropout(0.5, name='Bottleneck_Dropout')(c4)
    # Decoder
    u3 = UpSampling2D((2, 2), name='UpSample3')(c4)
    u3 = concatenate([u3, c3], name='Concat3')
    c5 = Conv2D(256, 3, activation='relu', padding='same', name='Conv5_1')(u3)
    c5 = Conv2D(256, 3, activation='relu', padding='same', name='Conv5_2')(c5)
    u2 = UpSampling2D((2, 2), name='UpSample2')(c5)
    u2 = concatenate([u2, c2], name='Concat2')
    c6 = Conv2D(128, 3, activation='relu', padding='same', name='Conv6_1')(u2)
    c6 = Conv2D(128, 3, activation='relu', padding='same', name='Conv6_2')(c6)
    u1 = UpSampling2D((2, 2), name='UpSample1')(c6)
    u1 = concatenate([u1, c1], name='Concat1')
    c7 = Conv2D(64, 3, activation='relu', padding='same', name='Conv7_1')(u1)
    c7 = Conv2D(64, 3, activation='relu', padding='same', name='Conv7_2')(c7)
    outputs = Conv2D(1, 1, activation='sigmoid', name='Output_Layer')(c7)
    return Model(inputs, outputs, name='TumorSegNet')

In [ ]:
# Instantiate and compile model
model = TumorSegNet(input_shape=(RESIZE, RESIZE, 3))
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient, 'accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    fill_mode='nearest'
)

# TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=0
)

# Custom callback to log sample predictions to TensorBoard
class PredictionLogger(tf.keras.callbacks.Callback):
    def __init__(self, val_images, val_masks):
        super(PredictionLogger, self).__init__()
        self.val_images = val_images[:5]  # Log first 5 validation images
        self.val_masks = val_masks[:5]

    def on_epoch_end(self, epoch, logs=None):
        preds = self.model.predict(self.val_images)
        preds = (preds > 0.5).astype(np.uint8)
        
        # Create figure with images, true masks, and predictions
        fig = plt.figure(figsize=(15, 5 * len(self.val_images)))
        for i in range(len(self.val_images)):
            plt.subplot(len(self.val_images), 3, i * 3 + 1)
            plt.imshow(self.val_images[i].squeeze(), cmap='gray')
            plt.title('MRI Image')
            plt.axis('off')
            plt.subplot(len(self.val_images), 3, i * 3 + 2)
            plt.imshow(self.val_masks[i].squeeze(), cmap='gray')
            plt.title('True Mask')
            plt.axis('off')
            plt.subplot(len(self.val_images), 3, i * 3 + 3)
            plt.imshow(preds[i].squeeze(), cmap='gray')
            plt.title(f'Predicted Mask (Epoch {epoch + 1})')
            plt.axis('off')
        
        # Log figure to TensorBoard
        writer = tf.summary.create_file_writer(log_dir)
        with writer.as_default():
            tf.summary.image("Validation Predictions", plot_to_image(fig), step=epoch)
        plt.close(fig)

# Convert matplotlib figure to TensorBoard image format
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

# Other callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint('TumorSegNet_best.keras', save_best_only=True, monitor='val_dice_coefficient'),
    tensorboard_callback,
    # PredictionLogger(x_val['img'], masks_val)
]

# Train the model
history = model.fit(
    trds,
    validation_data=vlds,
    epochs=3,
    callbacks=callbacks
)

Epoch 1/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 2995s 17s/step - accuracy: 0.9870 - dice_coefficient: 0.2113 - loss: 0.8516 - val_accuracy: 0.9912 - val_dice_coefficient: 0.4440 - val_loss: 0.6029
Epoch 2/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 3259s 19s/step - accuracy: 0.9886 - dice_coefficient: 0.4639 - loss: 0.5896 - val_accuracy: 0.9915 - val_dice_coefficient: 0.4534 - val_loss: 0.6011
Epoch 3/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 3219s 19s/step - accuracy: 0.9898 - dice_coefficient: 0.5064 - loss: 0.5423 - val_accuracy: 0.9915 - val_dice_coefficient: 0.4789 - val_loss: 0.5691
